## MERGE

In [1]:
import numpy as np
import pandas as pd
import os

path = os.getcwd()

In [2]:
df_census = pd.read_csv(os.path.join(path, 'csv/census_by_city.csv'))     # (25235, 35)
df_crime = pd.read_csv(os.path.join(path, 'csv/fbi_crime_uscities.csv'))  # (8103, 16)
df_rent = pd.read_csv(os.path.join(path, 'csv/rent_cleaned.csv'))         # (2831, 4)
df_poll = pd.read_csv(os.path.join(path, 'csv/pollution_summary.csv'))    # (845, 20)
df_lat_lon = pd.read_excel(os.path.join(path, 'csv/uscities.xlsx'))       # (28338, 17)

In [3]:
# replace 'State_Id' in df_census so it matches the other columns
df_census.rename(columns = {'State_Id':'State'}, inplace = True)
df_lat_lon.rename(columns = {'state_id':'State','city': 'City'}, inplace = True)
df_lat_lon = df_lat_lon[['City', 'State', 'lat', 'lng']]

In [4]:
# put all the dfs in a list
df_lst = [df_census, df_crime, df_rent, df_poll, df_lat_lon]

# remove whitespace in all the dfs
for x in df_lst:
    x['City'] = x['City'].str.strip()
    x['State'] = x['State'].str.strip()

# make sure it worked    
for x in df_lst:
    print(x['City'][0])
    print(x['State'][0])

Anchor Point
AK
Anchorage
AK
Aleutians East
AK
Anchorage
AK
Adak
AK


In [5]:
# merging the data and following its shape sequentially
df_net = pd.merge(df_census, df_crime, on=['City', 'State'])
print(df_net.shape)
df_net2 = pd.merge(df_net, df_rent, on=['City', 'State'])
print(df_net2.shape)
df_net3 = pd.merge(df_net2, df_poll, on=['City', 'State'])
print(df_net3.shape)
df_net4 = pd.merge(df_net3, df_lat_lon, on=['City', 'State'])
print(df_net4.shape)

(6866, 49)
(768, 51)
(431, 69)
(425, 71)


In [6]:
df_net4 = df_net4.drop_duplicates()

## CLEANUP

### Update Crime Ratings 
- comparison is across fewer cities. 
- run pd.qcut again

In [7]:
df_net4['Crime Rating'].value_counts()

High      330
Medium     84
Low         8
Name: Crime Rating, dtype: int64

crime rating skews very high because after the merging there were only a little over 400 cities versus the 8000 cities reported in the FBI crime data. 

In [8]:
# dropping old crime rating column
df_net4 = df_net4.drop(columns = ['Crime Rating'])

In [9]:
# Create a new crime rating by using qcut() on the number of merged cities we have
df_net4['Crime Rating'] = pd.qcut(df_net4['Crime Rate (per 1000 residents)'], q=3, labels=['Low', 'Medium', 'High'])

In [10]:
df_net4['Crime Rating'].value_counts()

High      141
Low       141
Medium    140
Name: Crime Rating, dtype: int64

### Clean Columns

In [11]:
col = ["Poverty", "ChildPoverty", "Unemployment", "Professional", "Office", "Service", "Construction", "Production"]
df_net4[col] = df_net4[col].round(2)

### Add an Index Column

In [12]:
df_net4['Index'] = df_net4.index

### Reorder Columns

In [13]:
df_net4.columns

Index(['City', 'State', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
       'Black', 'Native', 'Asian', 'Pacific', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Employed', 'Unemployment', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Unnamed: 0_x', 'Population',
       'Violent crime', 'Murder and nonnegligent manslaughter', 'Rape',
       'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',
       'Larceny- theft', 'Motor vehicle theft', 'Arson',
       'Crime Rate (per 1000 residents)', 'Unnamed: 0_y', 'Rent', 'Year',
       'Days with AQI', 'Good Days', 'Moderate Days',
       'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
       'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 

In [14]:
df_net4 = df_net4[[ 
    'Index',
    'City', 'State', 
    'lat', 'lng',
    'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
    'Black', 'Native', 'Asian', 'Pacific', 'Income', 'IncomeErr','IncomePerCap', 
    'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Employed', 'Unemployment', 
    'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Professional', 
    'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 
    'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 
    'Rent', 'Year',
    'Population','Violent crime', 'Murder and nonnegligent manslaughter', 'Rape',
    'Robbery', 'Aggravated assault', 'Property crime', 'Burglary','Larceny- theft', 
    'Motor vehicle theft', 'Arson', 'Crime Rate (per 1000 residents)', 'Crime Rating',
    'Days with AQI', 'Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days', 
    'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI', 
    '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2', 'Days Ozone', 'Days SO2', 
    'Days PM2.5', 'Days PM10', 'Level of Concern',
    ]]

In [15]:
df_net4.rename(columns = {'Level of Concern':'Air Quality Index'}, inplace = True)


In [16]:
pd.set_option('display.max_columns', None)

In [17]:
df_net4.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
# extract the merged df as a .csv
df_net4.to_csv('merged.csv', index = False) 